In [1]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect

import pandas as pd
import numpy as np


In [6]:
file_path = "../Nelson/Active_roster_stats.csv"

df = pd.read_csv(file_path)
df = df.copy()
df.head(20)
# len(df) 6930

,Unnamed: 0,Season,Age,Tm,Lg,Pos,G,GS,MP,FG,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Player
0,0,2016-17,20.0,BOS,NBA,SF,78.0,20.0,1341.0,192.0,...,45.0,175.0,220.0,64.0,35.0,18.0,68.0,142.0,515.0,Carsen Edwards
1,1,2017-18,21.0,BOS,NBA,SG,70.0,70.0,2152.0,373.0,...,66.0,280.0,346.0,114.0,70.0,26.0,124.0,181.0,1017.0,Carsen Edwards
2,2,2018-19,22.0,BOS,NBA,SG,74.0,25.0,1913.0,368.0,...,65.0,248.0,313.0,100.0,69.0,32.0,99.0,186.0,964.0,Carsen Edwards
3,3,2019-20,23.0,BOS,NBA,SG,3.0,3.0,86.0,20.0,...,1.0,19.0,20.0,6.0,2.0,3.0,4.0,11.0,52.0,Carsen Edwards
4,4,Career,NaN,NaN,NBA,NaN,225.0,118.0,5492.0,953.0,...,177.0,722.0,899.0,284.0,176.0,79.0,295.0,520.0,2548.0,Carsen Edwards
5,5,2019-20,21.0,BOS,NBA,PG,4.0,0.0,52.0,5.0,...,2.0,4.0,6.0,4.0,2.0,0.0,0.0,3.0,16.0,Tacko Fall
6,6,Career,NaN,NaN,NBA,NaN,4.0,0.0,52.0,5.0,...,2.0,4.0,6.0,4.0,2.0,0.0,0.0,3.0,16.0,Tacko Fall
7,7,2019-20,24.0,BOS,NBA,C,1.0,0.0,4.0,2.0,...,0.0,3.0,3.0,0.0,0.0,0.0,1.0,2.0,4.0,Javonte Green
8,8,Career,NaN,NaN,NBA,NaN,1.0,0.0,4.0,2.0,...,0.0,3.0,3.0,0.0,0.0,0.0,1.0,2.0,4.0,Javonte Green
9,9,2019-20,26.0,BOS,NBA,SG,3.0,0.0,4.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,Gordon Hayward


In [14]:
df = df[df["Season"].notnull()]
df = df[df["Season"].str.contains('-')]
df = df[~df["Season"].str.contains('-20')]
# len(df)  3783
df.head(20)

,Unnamed: 0,Season,Age,Tm,Lg,Pos,G,GS,MP,FG,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Player
0,0,2016-17,20.0,BOS,NBA,SF,78.0,20.0,1341.0,192.0,...,45.0,175.0,220.0,64.0,35.0,18.0,68.0,142.0,515.0,Carsen Edwards
1,1,2017-18,21.0,BOS,NBA,SG,70.0,70.0,2152.0,373.0,...,66.0,280.0,346.0,114.0,70.0,26.0,124.0,181.0,1017.0,Carsen Edwards
2,2,2018-19,22.0,BOS,NBA,SG,74.0,25.0,1913.0,368.0,...,65.0,248.0,313.0,100.0,69.0,32.0,99.0,186.0,964.0,Carsen Edwards
11,11,2010-11,20.0,UTA,NBA,SF,72.0,17.0,1218.0,142.0,...,41.0,98.0,139.0,81.0,30.0,19.0,70.0,106.0,388.0,Enes Kanter
12,12,2011-12,21.0,UTA,NBA,SF,66.0,58.0,2015.0,267.0,...,59.0,171.0,230.0,207.0,53.0,41.0,109.0,106.0,782.0,Enes Kanter
13,13,2012-13,22.0,UTA,NBA,SF,72.0,27.0,2104.0,336.0,...,51.0,174.0,225.0,213.0,58.0,38.0,120.0,125.0,1017.0,Enes Kanter
14,14,2013-14,23.0,UTA,NBA,SF,77.0,77.0,2800.0,426.0,...,62.0,329.0,391.0,400.0,110.0,40.0,212.0,155.0,1248.0,Enes Kanter
15,15,2014-15,24.0,UTA,NBA,SF,76.0,76.0,2618.0,484.0,...,54.0,319.0,373.0,313.0,108.0,30.0,206.0,131.0,1463.0,Enes Kanter
16,16,2015-16,25.0,UTA,NBA,SF,80.0,80.0,2893.0,521.0,...,61.0,336.0,397.0,296.0,95.0,27.0,202.0,183.0,1578.0,Enes Kanter
17,17,2016-17,26.0,UTA,NBA,SF,73.0,73.0,2516.0,545.0,...,49.0,345.0,394.0,252.0,73.0,20.0,140.0,117.0,1601.0,Enes Kanter


In [10]:
engine = create_engine("sqlite:///NBA.sqlite")

df.to_sql(name='Actives', con=engine, if_exists='replace', index=False)

In [11]:
pd.read_sql_query('SELECT * FROM Actives', con=engine).head()

,Unnamed: 0,Season,Age,Tm,Lg,Pos,G,GS,MP,FG,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Player
0,0,2016-17,20.0,BOS,NBA,SF,78.0,20.0,1341.0,192.0,...,45.0,175.0,220.0,64.0,35.0,18.0,68.0,142.0,515.0,Carsen Edwards
1,1,2017-18,21.0,BOS,NBA,SG,70.0,70.0,2152.0,373.0,...,66.0,280.0,346.0,114.0,70.0,26.0,124.0,181.0,1017.0,Carsen Edwards
2,2,2018-19,22.0,BOS,NBA,SG,74.0,25.0,1913.0,368.0,...,65.0,248.0,313.0,100.0,69.0,32.0,99.0,186.0,964.0,Carsen Edwards
3,3,2019-20,23.0,BOS,NBA,SG,3.0,3.0,86.0,20.0,...,1.0,19.0,20.0,6.0,2.0,3.0,4.0,11.0,52.0,Carsen Edwards
4,5,2019-20,21.0,BOS,NBA,PG,4.0,0.0,52.0,5.0,...,2.0,4.0,6.0,4.0,2.0,0.0,0.0,3.0,16.0,Tacko Fall
